In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re
from rapidfuzz import fuzz, process

In [2]:
df = pd.read_csv(r"D:\CRIME DATA SET\DATA\Project\data\police-shootings-export.csv")
df2 = pd.read_csv(r"D:\CRIME DATA SET\DATA\Project\data\Full_MedianHouseHoldIncome2015-2023.csv"  ,encoding='latin-1')
df3 = pd.read_csv(r"D:\CRIME DATA SET\DATA\Project\data\FULL_PeopleBelowPovertyLevel2015-2023.csv" ,encoding='latin-1')
df4 = pd.read_csv(r"D:\CRIME DATA SET\DATA\Project\data\Full_PrecentOfPeopleOver25CompletedHighSchool2015-2023.csv",encoding='latin-1' , low_memory=False )
df5 = pd.read_csv(r"D:\CRIME DATA SET\DATA\Project\data\Full_Race2015-2023.csv",encoding='latin-1')
States = pd.read_csv(r"D:\CRIME DATA SET\DATA\Project\data\States_in_America.csv",encoding='latin-1')
Matched = pd.read_csv(r"D:\PSUSA Project\data\matched_places.csv") #made by AI

# Matching the FACT and DIM

In [3]:
dfc = pd.merge(df , Matched , how="left" )
dfc

,date,name,age,gender,armed,race,city,state,flee,body_camera,signs_of_mental_illness,police_departments_involved,Correction,Census_Name,State
0,2015-01-02,Lewis Lee Lembke,47.0,male,gun,White,Aloha,OR,not,False,False,"Washington County Sheriff's Office, OR",Correction needed,Aloha CDP,Oregon
1,2015-01-02,Tim Elliot,53.0,male,gun,Asian,Shelton,WA,not,False,True,"Mason County Sheriff's Office, WA",Correction needed,Shelton city,Washington
2,2015-01-03,John Paul Quintero,23.0,male,unarmed,Hispanic,Wichita,KS,not,False,False,"Wichita Police Department, KS",Correction needed,Wichita city,Kansas
3,2015-01-04,Kenneth Joe Brown,18.0,male,gun,White,Guthrie,OK,not,False,False,"Oklahoma Highway Patrol, OK",Correction needed,Guthrie city,Oklahoma
4,2015-01-04,Michael Rodriguez,39.0,male,other,Hispanic,Evans,CO,not,False,False,"Evans Police Department, CO",No close match found,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9888,2024-07-13,Daniel Scott McGoldrick,35.0,male,gun,Unknown,Easley,SC,not,False,False,"Easley Police Department, SC",Correction needed,Easley city,South Carolina
9889,2024-07-15,NaN,NaN,male,undetermined,Unknown,Graham,WA,other,False,False,"Pierce County Sheriff's Department, WA",Correction needed,Graham CDP,Washington
9890,2024-07-15,Steven Kissack,35.0,male,knife,White,Juneau,AK,foot,False,False,"Alaska Wildlife Troopers, AK;Juneau Police Dep...",No close match found,NaN,NaN
9891,2024-07-15,Charles Patrick Carroll,68.0,male,replica,White,Beaumont,TX,not,False,True,"Beaumont Police Department, TX",Correction needed,Beaumont city,Texas


In [4]:
dfc['Census_Name'].fillna(dfc['city'] , inplace=True) 

In [5]:
dfc['city'].isnull().sum()

72

In [6]:
dfc['city'] = dfc['Census_Name']

In [7]:
dfc.drop(['Census_Name' , "Correction" , "State"] , axis=1 , inplace=True)

In [8]:
df = dfc

In [9]:
df2 = pd.merge(df2 , df3 , on=['year' , 'City' , 'State'] , how='outer')
df2 = pd.merge(df2 , df4 , on=['year' , 'City' , 'State'] , how='outer')
df2 = pd.merge(df2 , df5 , on=['year' , 'City' , 'State'] , how='outer')

In [10]:
df

,date,name,age,gender,armed,race,city,state,flee,body_camera,signs_of_mental_illness,police_departments_involved
0,2015-01-02,Lewis Lee Lembke,47.0,male,gun,White,Aloha CDP,OR,not,False,False,"Washington County Sheriff's Office, OR"
1,2015-01-02,Tim Elliot,53.0,male,gun,Asian,Shelton city,WA,not,False,True,"Mason County Sheriff's Office, WA"
2,2015-01-03,John Paul Quintero,23.0,male,unarmed,Hispanic,Wichita city,KS,not,False,False,"Wichita Police Department, KS"
3,2015-01-04,Kenneth Joe Brown,18.0,male,gun,White,Guthrie city,OK,not,False,False,"Oklahoma Highway Patrol, OK"
4,2015-01-04,Michael Rodriguez,39.0,male,other,Hispanic,Evans,CO,not,False,False,"Evans Police Department, CO"
...,...,...,...,...,...,...,...,...,...,...,...,...
9888,2024-07-13,Daniel Scott McGoldrick,35.0,male,gun,Unknown,Easley city,SC,not,False,False,"Easley Police Department, SC"
9889,2024-07-15,NaN,NaN,male,undetermined,Unknown,Graham CDP,WA,other,False,False,"Pierce County Sheriff's Department, WA"
9890,2024-07-15,Steven Kissack,35.0,male,knife,White,Juneau,AK,foot,False,False,"Alaska Wildlife Troopers, AK;Juneau Police Dep..."
9891,2024-07-15,Charles Patrick Carroll,68.0,male,replica,White,Beaumont city,TX,not,False,True,"Beaumont Police Department, TX"


In [11]:
cities  = df[df['city'].isnull()]
cities

,date,name,age,gender,armed,race,city,state,flee,body_camera,signs_of_mental_illness,police_departments_involved
2197,2017-03-20,Clarence Duane Huderle,73.0,male,gun,Unknown,NaN,MN,NaN,False,False,"Polk County Sheriff's Office, MN"
8005,2022-12-02,Jonathan Wiseman,39.0,male,gun,White,NaN,DE,other,False,False,"Delaware State Police, DE"
8027,2022-12-09,Michael Fredric Stevens,57.0,male,gun,White,NaN,VA,not,False,False,"Rockbridge County Sheriff's Office, VA"
8048,2022-12-17,NaN,NaN,NaN,gun,Unknown,NaN,WV,NaN,False,False,"McDowell County Sheriff's Office, WV"
8054,2022-12-19,Michael Cline,35.0,male,knife,White,NaN,VA,foot,False,False,"Louisa County Sheriff's Office, VA"
...,...,...,...,...,...,...,...,...,...,...,...,...
9197,2023-12-16,NaN,NaN,male,gun,Unknown,NaN,VA,NaN,False,False,"Stafford County Sheriff's Department, VA"
9211,2023-12-20,David Estrada,38.0,male,gun,Unknown,NaN,CO,NaN,False,False,"Adams County Sheriff's Department, CO"
9225,2023-12-24,Efren Inda,50.0,male,gun,Hispanic,NaN,CO,NaN,False,False,"Boulder County Sheriff's Office, CO;Colorado S..."
9595,2024-04-27,Brian James Propes,42.0,male,gun,White,NaN,FL,not,False,False,"Washington County Sheriff's Office, FL"


In [12]:
def get_mode (data) :
    modes = data.mode()
    if not modes.empty :
        return modes[0]
    else :
        return np.nan

In [13]:
sorted = df[['city' , 'state' , 'police_departments_involved']].sort_values(by =['police_departments_involved'])
sorted

,city,state,police_departments_involved
6423,Abbeville city,SC,"Abbeville County Sheriff's Office, SC"
4598,Aberdeen city,WA,"Aberdeen Police Department, WA"
6474,Abilene city,TX,"Abilene Police Department, TX"
5150,Abilene city,TX,"Abilene Police Department, TX"
7506,Abilene city,TX,"Abilene Police Department, TX"
...,...,...,...
6150,Laredo city,TX,"Zapata County Sheriff's Department, TX"
255,Zion,IL,"Zion Police Department, IL"
1010,Zion,IL,"Zion Police Department, IL"
7746,Zion,IL,"Zion Police Department, IL"


In [14]:
df.groupby('police_departments_involved')['city'].apply(get_mode)

police_departments_involved
Abbeville County Sheriff's Office, SC         Abbeville city
Aberdeen Police Department, WA                 Aberdeen city
Abilene Police Department, TX                   Abilene city
Abington Township Police Department, PA    Abington Township
Acadia Parish Sheriff's Office, LA              Crowley city
                                                 ...        
Yuba County Sheriff's Office, CA              Olivehurst CDP
Yuma County Sheriff's Department, AZ                    Yuma
Yuma Police Department, AZ                              Yuma
Zapata County Sheriff's Department, TX           Laredo city
Zion Police Department, IL                              Zion
Name: city, Length: 3726, dtype: object

In [15]:
dict_of_cites = df.groupby('police_departments_involved')['city'].apply(get_mode).to_dict()

In [16]:
df['city'].fillna(df['police_departments_involved'].map(dict_of_cites) ,  inplace=True)

In [17]:
df['city'].isnull().sum()

38

In [18]:
df[df['city'].isnull()]

,date,name,age,gender,armed,race,city,state,flee,body_camera,signs_of_mental_illness,police_departments_involved
2197,2017-03-20,Clarence Duane Huderle,73.0,male,gun,Unknown,NaN,MN,NaN,False,False,"Polk County Sheriff's Office, MN"
8048,2022-12-17,NaN,NaN,NaN,gun,Unknown,NaN,WV,NaN,False,False,"McDowell County Sheriff's Office, WV"
8054,2022-12-19,Michael Cline,35.0,male,knife,White,NaN,VA,foot,False,False,"Louisa County Sheriff's Office, VA"
8094,2023-01-02,Jerry Preston,49.0,male,undetermined,White,NaN,KY,NaN,False,False,"Kentucky State Police, KY;Perry County Sheriff..."
8141,2023-01-13,Devin Cribbs,24.0,male,gun,White,NaN,AL,car,False,False,"Vernon Police Department, AL"
8269,2023-02-24,Erin Williamson,28.0,male,unknown,White,NaN,KY,foot,False,False,"Calloway County Sheriff's Department, KY"
8372,2023-04-01,Tyler Raymer,28.0,male,gun,Unknown,NaN,MO,NaN,False,False,"Morgan County Sheriff's Office, MO"
8392,2023-04-09,NaN,NaN,male,unknown,Unknown,NaN,MS,NaN,False,False,"Lincoln County Sheriff's Office, MS"
8411,2023-04-14,William E. Harver Sr.,45.0,male,gun,White,NaN,VA,not,False,False,"Amelia County Sheriff's Office, VA"
8426,2023-04-20,Dimitri Angel Amarillas,25.0,male,knife,Hispanic,NaN,TX,NaN,False,False,"Manor Police Department, TX;Travis County Sher..."


In [19]:
df[['state', 'city' , 'police_departments_involved']].sort_values(['police_departments_involved' , 'state'])

,state,city,police_departments_involved
6423,SC,Abbeville city,"Abbeville County Sheriff's Office, SC"
4598,WA,Aberdeen city,"Aberdeen Police Department, WA"
3198,TX,Abilene city,"Abilene Police Department, TX"
5150,TX,Abilene city,"Abilene Police Department, TX"
6474,TX,Abilene city,"Abilene Police Department, TX"
...,...,...,...
6150,TX,Laredo city,"Zapata County Sheriff's Department, TX"
255,IL,Zion,"Zion Police Department, IL"
1010,IL,Zion,"Zion Police Department, IL"
7746,IL,Zion,"Zion Police Department, IL"


In [20]:
cities_by_state_mode = df.groupby('state')['city'].transform(get_mode)

In [21]:
df['city'].fillna(cities_by_state_mode , inplace=True)

In [22]:
df['city'].isnull().sum()

0

In [23]:
df['date'] = pd.to_datetime(df['date'] , format = '%Y-%m-%d')

In [24]:
df['date'].isnull().sum()

0

In [25]:
df['year'] = df['date'].dt.year

In [26]:
States.drop('Id' , axis=1 , inplace=True)

In [27]:
States.rename(columns={'StateName' : 'State'} , inplace=True)

In [28]:
States[(States["Code"] =="FL") | (States["Code"] =="TN")]
#Village of islands village; florida	
#Moore county metropolitan government; tennessee
#Village of islands village	
#Moore county metropolitan government

,State,Code
9,Florida,FL
10,Village of Islands village; Florida,FL
43,Tennessee,TN
44,Moore County metropolitan government; Tennessee,TN
54,Village of Islands village,FL
55,Moore County metropolitan government,TN


In [29]:
#States = States[~(States["State"]=="Village of islands village; florida")  ]
#States =States[~(States["State"]=="Moore county metropolitan government; tennessee")  ]
#States =States[~(States["State"]=="Village of islands village")  ]
#States =States[~(States["State"]=="Moore county metropolitan government") ]
States.drop_duplicates(subset=["Code"] , inplace=True)

In [30]:
States

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [31]:
df2['State']=df2['State'].str.lower()
df2['State']=df2['State'].str.strip()
df2['State']=df2['State'].str.capitalize()

States['State'] =States['State'].str.lower()
States['State'] =States['State'].str.strip()
States['State'] =States['State'].str.capitalize()

In [32]:
df2 = pd.merge(df2 , States , how='left' )

In [33]:
len(df2)

276529

In [34]:
df.rename(columns={'state' : 'Code'} , inplace=True)

In [35]:
df.rename(columns={'city' : 'City'} , inplace=True)

In [36]:
df

,date,name,age,gender,armed,race,City,Code,flee,body_camera,signs_of_mental_illness,police_departments_involved,year
0,2015-01-02,Lewis Lee Lembke,47.0,male,gun,White,Aloha CDP,OR,not,False,False,"Washington County Sheriff's Office, OR",2015
1,2015-01-02,Tim Elliot,53.0,male,gun,Asian,Shelton city,WA,not,False,True,"Mason County Sheriff's Office, WA",2015
2,2015-01-03,John Paul Quintero,23.0,male,unarmed,Hispanic,Wichita city,KS,not,False,False,"Wichita Police Department, KS",2015
3,2015-01-04,Kenneth Joe Brown,18.0,male,gun,White,Guthrie city,OK,not,False,False,"Oklahoma Highway Patrol, OK",2015
4,2015-01-04,Michael Rodriguez,39.0,male,other,Hispanic,Evans,CO,not,False,False,"Evans Police Department, CO",2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9888,2024-07-13,Daniel Scott McGoldrick,35.0,male,gun,Unknown,Easley city,SC,not,False,False,"Easley Police Department, SC",2024
9889,2024-07-15,NaN,NaN,male,undetermined,Unknown,Graham CDP,WA,other,False,False,"Pierce County Sheriff's Department, WA",2024
9890,2024-07-15,Steven Kissack,35.0,male,knife,White,Juneau,AK,foot,False,False,"Alaska Wildlife Troopers, AK;Juneau Police Dep...",2024
9891,2024-07-15,Charles Patrick Carroll,68.0,male,replica,White,Beaumont city,TX,not,False,True,"Beaumont Police Department, TX",2024


In [37]:
df2

,year,State,City,Median household income,PercentOfPeopleBelowPovertyLevel,Percent high school graduate or higher,share_white,share_black,share_native,share_asian,share_hispanic,Code
0,2015,Alabama,Abanda CDP,11207,78.8,21.2,76.9,23.1,0.0,0.0,0.0,AL
1,2015,Alabama,Abbeville city,25615,29.1,69.1,52.8,46.4,0.0,0.0,1.0,AL
2,2015,Alabama,Adamsville city,42575,25.5,78.9,47.6,43.3,0.0,0.5,9.1,AL
3,2015,Alabama,Addison town,37083,30.7,81.4,94.0,0.0,0.3,0.0,0.0,AL
4,2015,Alabama,Akron town,21667,42.0,68.6,20.1,79.9,0.0,0.0,0.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...
276524,2023,Puerto rico,Santa BÃ¡rbara comunidad,NaN,NaN,NaN,13.2,18.2,0.0,0.0,100.0,PR
276525,2023,Puerto rico,Santa MarÃ­a comunidad,NaN,NaN,NaN,1.8,0.0,0.0,0.0,100.0,PR
276526,2023,Puerto rico,SuÃ¡rez comunidad,NaN,NaN,NaN,9.3,42.9,0.0,0.0,100.0,PR
276527,2023,Puerto rico,TomÃ¡s de Castro comunidad,NaN,NaN,NaN,73.1,0.0,0.0,0.0,100.0,PR


In [38]:
#first function 
def fuzzy_match_city(row, df2):
    code = row['Code']
    city = row['City']
    
    subset = df2[df2['Code'] == code]
    if subset.empty:
        return pd.Series([None, None])
    
    match = process.extractOne(
        city,
        subset['City'],
        scorer=fuzz.WRatio
    )
    
    if match:
        matched_city, score, _ = match
        return pd.Series([matched_city, score])
    else:
        return pd.Series([None, None])

df[['Matched_City', 'Match_Score']] = df.apply(lambda row: fuzzy_match_city(row, df2), axis=1)


In [39]:
df[df['Match_Score'] == 100].sort_values(by="Match_Score" , ascending=False)

,date,name,age,gender,armed,race,City,Code,flee,body_camera,signs_of_mental_illness,police_departments_involved,year,Matched_City,Match_Score
0,2015-01-02,Lewis Lee Lembke,47.0,male,gun,White,Aloha CDP,OR,not,False,False,"Washington County Sheriff's Office, OR",2015,Aloha CDP,100.0
6628,2021-08-24,NaN,31.0,male,unarmed,Unknown,Payson town,AZ,NaN,False,False,"Payson Police Department, AZ",2021,Payson town,100.0
6643,2021-08-27,Alexander Tadros,30.0,male,gun,White,Portland city,OR,not,False,False,"Portland Police Department, OR",2021,Portland city,100.0
6641,2021-08-27,Lonnel Cephas Jr.,19.0,male,gun,Unknown,Elkhart city,IN,car,False,False,"Elkhart Police Department, IN",2021,Elkhart city,100.0
6640,2021-08-27,Fanta Bility,8.0,female,unarmed,Black,Sharon Hill borough,PA,not,False,False,"Sharon Hill Police Department, PA",2021,Sharon Hill borough,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,2018-05-06,David P. Wolosin,38.0,male,gun,White,Casper city,WY,not,False,False,"Casper Police Department, WY",2018,Casper city,100.0
3334,2018-05-05,Mitchell Simmons,24.0,male,gun,White,Kettering city,OH,not,False,False,"Kettering Police Department, OH",2018,Kettering city,100.0
3333,2018-05-05,Stephen Charles Arthur,51.0,male,gun,White,Scottsdale city,AZ,foot,False,False,"Scottsdale Police Department, AZ",2018,Scottsdale city,100.0
3331,2018-05-05,Savannah Hill,21.0,female,vehicle,White,Aurora city,MO,car,False,False,"Missouri Highway Patrol, MO",2018,Aurora city,100.0


In [40]:
len(df[df['Match_Score'] > 85])

9305

In [41]:
df["Match_Score"].mean()

96.48085060727489

In [42]:
df["Match_Score"].median()

100.0

In [43]:
df["Match_Score"].mode()

0    100.0
Name: Match_Score, dtype: float64

In [44]:
df["Match_Score"].describe()

count    9893.000000
mean       96.480851
std         9.131043
min        40.000000
25%       100.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: Match_Score, dtype: float64

In [45]:
df[df["Match_Score"] != 100]["Match_Score"].describe()

count    1858.000000
mean       81.262139
std        12.602115
min        40.000000
25%        72.000000
50%        90.000000
75%        90.000000
max        95.000000
Name: Match_Score, dtype: float64

In [46]:
df["City"] = df["Matched_City"]

In [47]:
df.drop(["Matched_City" , "Match_Score"] , axis=1 , inplace=True)

# Cleaning Fact Table

#

# Cleaning Name Column

In [48]:
df['name'].isnull().sum()

337

In [49]:
df['name'].fillna('unknown' , inplace=True)

In [50]:
df['name'].isnull().sum()

0

# Cleaning age Column

In [51]:
df['age'].isnull().sum()

384

In [52]:
df['age'].mean()

37.40950678304764

In [53]:
df['age'].median()

35.0

In [54]:
df['age'].mode()

0    33.0
Name: age, dtype: float64

In [55]:
df['age'].fillna(df['age'].median() , inplace=True)

In [56]:
df['age'].sort_values().head(10)

7237     2.0
9463     4.0
2906     6.0
834      6.0
6640     8.0
7165    12.0
1015    12.0
7456    13.0
1681    13.0
9840    13.0
Name: age, dtype: float64

In [57]:
df['age'] =df['age'].astype(int)

In [58]:
Q1=df['age'].quantile(0.25)
Q2=df['age'].quantile(0.5)
Q3=df['age'].quantile(0.75)
IQR = Q3 - Q1
upper = Q3 + IQR * 1.5
lower= Q1 - IQR * 1.5

df[(df['age'] > upper)  | (df['age'] < lower)]
##this data is not a mistake its a real accedent happend with childrens and adults

,date,name,age,gender,armed,race,City,Code,flee,body_camera,signs_of_mental_illness,police_departments_involved,year
20,2015-01-09,Jimmy Foreman,71,male,gun,White,England city,AR,not,False,False,"England Police Department, AR",2015
28,2015-01-14,Talbot Schroeder,75,male,knife,White,Old Bridge CDP,NJ,not,False,False,"Old Bridge Police Department, NJ",2015
91,2015-02-07,James Allen,74,male,gun,Black,Gastonia city,NC,not,False,False,"Gastonia Police Department, NC",2015
125,2015-02-20,Douglas Harris,77,male,gun,Black,Birmingham city,AL,not,False,True,"Birmingham Police Department, AL",2015
275,2015-04-12,Richard Dale Weaver,83,male,knife,White,Oklahoma City city,OK,not,False,True,"Oklahoma City Police Department, OK",2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9308,2024-01-22,Janet Sours,81,female,knife,White,Wildwood city,FL,not,False,True,"Sumter County Sheriff's Department, FL",2024
9387,2024-02-18,Wendall Cross,85,male,gun,White,Blairsville city,GA,not,False,False,"Union County Sheriff's Office, GA",2024
9678,2024-05-14,Aubrey James Osteen,77,male,gun,Unknown,Tucumcari city,NM,not,False,False,"Quay County Sheriff's Office, NM",2024
9694,2024-05-17,Eugene Mewes,80,male,gun,Unknown,East Moline city,IL,NaN,False,False,"East Moline Police Department, IL",2024


# Cleaning gender Column

In [59]:
df['gender'].isnull().sum()

28

In [60]:
(df['gender'] == 'male').sum()/len(df)*100

95.2289497624583

In [61]:
df['gender'].fillna(df['gender'].mode()[0] , inplace=True)

In [62]:
(df['gender'] == 'male').sum()/len(df)*100

95.51197816638027

# Cleaning armed Column

In [63]:
for tool in df['armed'].unique() :
    print(tool)

gun
unarmed
other
replica
knife
blunt_object
nan
vehicle
undetermined
other,gun
unknown
blunt_object,blunt_object
gun,knife
knife,blunt_object
vehicle,gun
gun,vehicle
replica,vehicle
blunt_object,knife
knife,vehicle
vehicle,knife,other
knife,knife
replica,knife
other,blunt_object,knife
other,knife
vehicle,knife
gun,other
blunt_object,other


In [64]:
#  undetermined = unknown = nan
#  blunt_object,blunt_object = blunt_object
#  knife,knife = knife
#  knife,vehicle = vehicle,knife
#  other,gun = gun,other
#  blunt_object,knife = knife,blunt_object

In [65]:
round(df['armed'].isnull().sum()/len(df)*100 , 2)

2.13

In [66]:
df['armed'].replace(['undetermined' , 'unknown' ] , np.nan , inplace=True)

In [67]:
df['armed'].isnull().sum()/len(df)*100

7.884362680683312

In [68]:
df.groupby('armed')['armed'].count().sort_values(ascending = False)

armed
gun                          5748
knife                        1683
unarmed                       551
vehicle                       351
replica                       314
blunt_object                  239
other                          98
gun,knife                      40
gun,vehicle                    38
vehicle,gun                    20
knife,knife                     5
knife,blunt_object              4
other,gun                       4
knife,vehicle                   3
blunt_object,blunt_object       3
blunt_object,knife              3
vehicle,knife                   2
blunt_object,other              1
other,knife                     1
replica,vehicle                 1
replica,knife                   1
gun,other                       1
other,blunt_object,knife        1
vehicle,knife,other             1
Name: armed, dtype: int64

In [69]:
df['armed'].replace('vehicle,gun' , 'gun,vehicle' ,  inplace=True)
df['armed'].replace('vehicle,knife' , 'knife,vehicle' ,  inplace=True)
df['armed'].replace('blunt_object,knife' , 'knife,blunt_object' ,  inplace=True)
df['armed'].replace('blunt_object,blunt_object' , 'blunt_object' ,  inplace=True)
df['armed'].replace('knife,knife' , 'knife' ,  inplace=True)
df['armed'] = df['armed'].str.replace(',' , ' & ')

In [70]:
df.groupby('armed')['armed'].count().sort_values(ascending = False)

armed
gun                             5748
knife                           1688
unarmed                          551
vehicle                          351
replica                          314
blunt_object                     242
other                             98
gun & vehicle                     58
gun & knife                       40
knife & blunt_object               7
knife & vehicle                    5
other & gun                        4
blunt_object & other               1
other & blunt_object & knife       1
other & knife                      1
replica & knife                    1
replica & vehicle                  1
gun & other                        1
vehicle & knife & other            1
Name: armed, dtype: int64

In [71]:
df['armed'].isnull().sum()/len(df)*100

7.884362680683312

In [72]:
distribution =df['armed'].value_counts(normalize=True)

In [73]:
Number_of_nulls = df['armed'].isnull().sum()

In [74]:

# Fill missing 'armed' values by categories based on their original distribution

if  Number_of_nulls> 0:   
    fill_values = np.random.choice(distribution.index , size = Number_of_nulls , p= distribution.values)
    df.loc[df['armed'].isnull(), 'armed'] = fill_values

In [75]:
df['armed'].isnull().sum()

0

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9893 entries, 0 to 9892
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         9893 non-null   datetime64[ns]
 1   name                         9893 non-null   object        
 2   age                          9893 non-null   int32         
 3   gender                       9893 non-null   object        
 4   armed                        9893 non-null   object        
 5   race                         9893 non-null   object        
 6   City                         9893 non-null   object        
 7   Code                         9893 non-null   object        
 8   flee                         8548 non-null   object        
 9   body_camera                  9893 non-null   bool          
 10  signs_of_mental_illness      9893 non-null   bool          
 11  police_departments_involved  9892 non-null 

# Cleaning race column

In [77]:
x=0
for i in df['race'].unique() :
    x+=1
    print(x , i )

1 White
2 Asian
3 Hispanic
4 Black
5 Other
6 Unknown
7 Native American
8 White,Black,Native American
9 Native American,Hispanic
10 White,Hispanic
11 Black,Hispanic
12 White,Black


In [78]:
df['race'].value_counts()

race
White                          4432
Black                          2346
Hispanic                       1623
Unknown                        1144
Asian                           175
Native American                 135
Other                            31
Black,Hispanic                    2
White,Black                       2
White,Black,Native American       1
Native American,Hispanic          1
White,Hispanic                    1
Name: count, dtype: int64

# Cleaning State Column

In [79]:
df['Code'].nunique()

51

In [80]:
df['Code'].unique()

array(['OR', 'WA', 'KS', 'OK', 'CO', 'CA', 'AZ', 'PA', 'IA', 'LA', 'OH',
       'TX', 'UT', 'AR', 'MT', 'NV', 'IL', 'NM', 'NJ', 'VA', 'MO', 'MN',
       'IN', 'KY', 'MA', 'NH', 'FL', 'MD', 'ID', 'NE', 'MI', 'GA', 'TN',
       'NC', 'NY', 'AK', 'ME', 'AL', 'MS', 'WI', 'SC', 'DE', 'DC', 'WV',
       'HI', 'WY', 'ND', 'CT', 'SD', 'VT', 'RI'], dtype=object)

In [81]:
df['Code'].isnull().sum()

0

In [82]:
df['Code'] =df['Code'].str.upper()
df['Code']

0       OR
1       WA
2       KS
3       OK
4       CO
        ..
9888    SC
9889    WA
9890    AK
9891    TX
9892    IN
Name: Code, Length: 9893, dtype: object

# Cleaning Flee column

In [83]:
df['flee'].isnull().sum()/len(df)*100

13.595471545537249

In [84]:
df['flee'].unique()

array(['not', 'car', 'foot', 'other', nan], dtype=object)

In [85]:
df['flee'].value_counts(normalize=True)

flee
not      0.623304
car      0.182148
foot     0.151614
other    0.042934
Name: proportion, dtype: float64

In [86]:
df.loc[df['flee'].isnull() , 'flee'] = 'Unknown'

In [87]:
df['flee'].value_counts(normalize=True)

flee
not        0.538563
car        0.157384
Unknown    0.135955
foot       0.131002
other      0.037097
Name: proportion, dtype: float64

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9893 entries, 0 to 9892
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         9893 non-null   datetime64[ns]
 1   name                         9893 non-null   object        
 2   age                          9893 non-null   int32         
 3   gender                       9893 non-null   object        
 4   armed                        9893 non-null   object        
 5   race                         9893 non-null   object        
 6   City                         9893 non-null   object        
 7   Code                         9893 non-null   object        
 8   flee                         9893 non-null   object        
 9   body_camera                  9893 non-null   bool          
 10  signs_of_mental_illness      9893 non-null   bool          
 11  police_departments_involved  9892 non-null 

# Cleaning bolean columns (body_camera , signs_of_mental_illness)

In [89]:
df['body_camera'].unique()

array([False,  True])

In [90]:
df['signs_of_mental_illness'].unique()

array([False,  True])

# Cleaning police_departments_involved Column

In [91]:
df['police_departments_involved'].isnull().sum()

1

In [92]:
df[df["police_departments_involved"].isnull()]

,date,name,age,gender,armed,race,City,Code,flee,body_camera,signs_of_mental_illness,police_departments_involved,year
9770,2024-06-10,Ridge Ryen Rhodes,29,male,gun,White,Lewistown borough,PA,not,False,False,NaN,2024


In [93]:
df[df['City'] == 'Lewistown borough']

,date,name,age,gender,armed,race,City,Code,flee,body_camera,signs_of_mental_illness,police_departments_involved,year
2278,2017-04-26,Charles Bossinger,53,male,gun,White,Lewistown borough,PA,not,False,True,"Lewistown Borough Police Department, PA",2017
9770,2024-06-10,Ridge Ryen Rhodes,29,male,gun,White,Lewistown borough,PA,not,False,False,NaN,2024


In [94]:
df.loc[df['police_departments_involved'].isnull() , 'police_departments_involved'] ='Lewistown Borough Police Department, PA'

In [95]:
df['police_departments_involved'].isnull().sum()

0

# Cleaning Diminsions

In [96]:
df2.head()

,year,State,City,Median household income,PercentOfPeopleBelowPovertyLevel,Percent high school graduate or higher,share_white,share_black,share_native,share_asian,share_hispanic,Code
0,2015,Alabama,Abanda CDP,11207,78.8,21.2,76.9,23.1,0.0,0.0,0.0,AL
1,2015,Alabama,Abbeville city,25615,29.1,69.1,52.8,46.4,0.0,0.0,1.0,AL
2,2015,Alabama,Adamsville city,42575,25.5,78.9,47.6,43.3,0.0,0.5,9.1,AL
3,2015,Alabama,Addison town,37083,30.7,81.4,94.0,0.0,0.3,0.0,0.0,AL
4,2015,Alabama,Akron town,21667,42.0,68.6,20.1,79.9,0.0,0.0,0.0,AL


In [97]:
df2 = df2.sort_values(by=['State', 'City', 'year'])

In [98]:
df2

,year,State,City,Median household income,PercentOfPeopleBelowPovertyLevel,Percent high school graduate or higher,share_white,share_black,share_native,share_asian,share_hispanic,Code
0,2015,Alabama,Abanda CDP,11207,78.8,21.2,76.9,23.1,0.0,0.0,0.0,AL
29574,2016,Alabama,Abanda CDP,-,27.2,8.7,91.4,8.6,0.0,0.0,0.0,AL
59148,2017,Alabama,Abanda CDP,-,25.9,8.4,86.2,13.8,0.0,0.0,0.0,AL
88724,2018,Alabama,Abanda CDP,-,36.2,24.4,79.8,20.2,0.0,0.0,0.0,AL
118297,2019,Alabama,Abanda CDP,24261,26.7,39,74.4,25.6,0.0,0.0,0.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...
147615,2019,Wyoming,Yoder town,38229,4.1,88.3,100.0,0.0,0.0,0.0,1.2,WY
179484,2020,Wyoming,Yoder town,38125,16.9,88.4,94.0,0.0,0.0,0.0,0.0,WY
211392,2021,Wyoming,Yoder town,32917,20.9,88.7,93.6,0.0,0.0,0.0,0.0,WY
243578,2022,Wyoming,Yoder town,27417,30.4,89.4,90.2,0.0,0.0,0.0,0.0,WY


In [99]:
columns_to_process = [
    'Median household income',
    'PercentOfPeopleBelowPovertyLevel',
    'Percent high school graduate or higher',
    'share_white',
    'share_black',
    'share_native',
    'share_asian',
    'share_hispanic'
]

In [100]:
#change columns to numric
for col in columns_to_process:
    df2[col] = pd.to_numeric(df2[col], errors='coerce')

In [101]:
#display nulls precntage
for col in columns_to_process:
    print(col , " = " ,round(df2[col].isnull().sum()/len(df2[col])*100 , 2 ), "%")

Median household income  =  9.21 %
PercentOfPeopleBelowPovertyLevel  =  1.27 %
Percent high school graduate or higher  =  1.22 %
share_white  =  1.19 %
share_black  =  1.19 %
share_native  =  1.19 %
share_asian  =  1.19 %
share_hispanic  =  1.19 %


In [102]:
#after sorting the data as before 
# For each column, fill missing values by interpolating within each unique State-City group.
for col in columns_to_process:
    df2[col] = df2.groupby(['State', 'City'])[col].transform(lambda x: x.interpolate(limit_direction='both'))


In [103]:
#display nulls precntage
for col in columns_to_process:
    print(col , " = " ,round(df2[col].isnull().sum()/len(df2[col])*100 , 2 ), "%")

Median household income  =  3.09 %
PercentOfPeopleBelowPovertyLevel  =  0.44 %
Percent high school graduate or higher  =  0.43 %
share_white  =  0.52 %
share_black  =  0.52 %
share_native  =  0.52 %
share_asian  =  0.52 %
share_hispanic  =  0.52 %


In [104]:
# Fill the remaining nulls with the median of each state for each year individually
for col in columns_to_process:
    median_fill_values = df2.groupby(['State', 'year'])[col].transform('median')
    df2[col].fillna(median_fill_values, inplace=True)

In [105]:
#display nulls precntage
for col in columns_to_process:
    print(col , " = " ,round(df2[col].isnull().sum()/len(df2[col])*100 , 2 ), "%")

Median household income  =  0.0 %
PercentOfPeopleBelowPovertyLevel  =  0.0 %
Percent high school graduate or higher  =  0.0 %
share_white  =  0.0 %
share_black  =  0.0 %
share_native  =  0.0 %
share_asian  =  0.0 %
share_hispanic  =  0.0 %


In [106]:
df2

,year,State,City,Median household income,PercentOfPeopleBelowPovertyLevel,Percent high school graduate or higher,share_white,share_black,share_native,share_asian,share_hispanic,Code
0,2015,Alabama,Abanda CDP,11207.0,78.8,21.2,76.9,23.1,0.0,0.0,0.0,AL
29574,2016,Alabama,Abanda CDP,14470.5,27.2,8.7,91.4,8.6,0.0,0.0,0.0,AL
59148,2017,Alabama,Abanda CDP,17734.0,25.9,8.4,86.2,13.8,0.0,0.0,0.0,AL
88724,2018,Alabama,Abanda CDP,20997.5,36.2,24.4,79.8,20.2,0.0,0.0,0.0,AL
118297,2019,Alabama,Abanda CDP,24261.0,26.7,39.0,74.4,25.6,0.0,0.0,0.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...
147615,2019,Wyoming,Yoder town,38229.0,4.1,88.3,100.0,0.0,0.0,0.0,1.2,WY
179484,2020,Wyoming,Yoder town,38125.0,16.9,88.4,94.0,0.0,0.0,0.0,0.0,WY
211392,2021,Wyoming,Yoder town,32917.0,20.9,88.7,93.6,0.0,0.0,0.0,0.0,WY
243578,2022,Wyoming,Yoder town,27417.0,30.4,89.4,90.2,0.0,0.0,0.0,0.0,WY


In [107]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276529 entries, 0 to 275903
Data columns (total 12 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   year                                    276529 non-null  int64  
 1   State                                   276529 non-null  object 
 2   City                                    276529 non-null  object 
 3   Median household income                 276529 non-null  float64
 4   PercentOfPeopleBelowPovertyLevel        276529 non-null  float64
 5   Percent high school graduate or higher  276529 non-null  float64
 6   share_white                             276529 non-null  float64
 7   share_black                             276529 non-null  float64
 8   share_native                            276529 non-null  float64
 9   share_asian                             276529 non-null  float64
 10  share_hispanic                          276529 no

In [108]:
columns_to_check = [
    'PercentOfPeopleBelowPovertyLevel',
    'Percent high school graduate or higher',
    'share_white',
    'share_black',
    'share_native',
    'share_asian',
    'share_hispanic'
]

for col in columns_to_check:
    outliers = df2[(df2[col] < 0) | (df2[col] > 100)]
    if not outliers.empty:
        print(f" there is wrong value in :  '{col}':")
        print(outliers[[col]]) 
    else:
        print(f" column '{col}'is clean no problems" )

print("Chek is done")

 column 'PercentOfPeopleBelowPovertyLevel'is clean no problems
 column 'Percent high school graduate or higher'is clean no problems
 column 'share_white'is clean no problems
 column 'share_black'is clean no problems
 column 'share_native'is clean no problems
 column 'share_asian'is clean no problems
 column 'share_hispanic'is clean no problems
Chek is done


In [109]:
df2.isnull().sum()

year                                       0
State                                      0
City                                       0
Median household income                    0
PercentOfPeopleBelowPovertyLevel           0
Percent high school graduate or higher     0
share_white                                0
share_black                                0
share_native                               0
share_asian                                0
share_hispanic                             0
Code                                      18
dtype: int64

In [118]:
df2[~df2["Code"].isnull()]

,year,State,City,Median household income,PercentOfPeopleBelowPovertyLevel,Percent high school graduate or higher,share_white,share_black,share_native,share_asian,share_hispanic,Code
174123,2020,Moore county metropolitan government,Lynchburg,63762.0,8.0,86.4,92.7,1.9,1.1,3.3,0.0,NaN
206029,2021,Moore county metropolitan government,Lynchburg,64926.0,8.0,86.7,92.6,1.7,0.8,3.2,0.2,NaN
238213,2022,Moore county metropolitan government,Lynchburg,69250.0,9.9,86.1,92.5,3.7,0.5,0.9,0.5,NaN
270520,2023,Moore county metropolitan government,Lynchburg,66687.0,11.3,89.1,92.2,2.9,0.4,0.4,1.6,NaN
24324,2015,Moore county metropolitan government; tennessee,Lynchburg,49427.0,10.8,87.2,94.4,3.9,1.1,0.0,0.4,NaN
53897,2016,Moore county metropolitan government; tennessee,Lynchburg,49496.0,10.4,86.1,93.9,3.5,1.5,0.0,0.3,NaN
83472,2017,Moore county metropolitan government; tennessee,Lynchburg,51671.0,7.7,87.3,93.5,1.7,1.6,2.0,0.2,NaN
113049,2018,Moore county metropolitan government; tennessee,Lynchburg,52294.0,7.9,85.6,93.3,1.3,2.4,1.7,0.2,NaN
142622,2019,Moore county metropolitan government; tennessee,Lynchburg,57708.0,7.7,85.9,92.8,2.2,1.6,2.4,0.2,NaN
152675,2020,Village of islands village,Islamorada,76497.0,5.9,96.3,88.9,0.3,0.2,1.2,13.8,NaN


In [120]:
df2.dropna(inplace=True)

In [126]:
df2.isnull().sum()

year                                      0
State                                     0
City                                      0
Median household income                   0
PercentOfPeopleBelowPovertyLevel          0
Percent high school graduate or higher    0
share_white                               0
share_black                               0
share_native                              0
share_asian                               0
share_hispanic                            0
Code                                      0
dtype: int64

In [121]:
df.duplicated().sum()

0

In [122]:
df2.duplicated(subset=['City', 'Code', 'year']).sum()

0

In [123]:
#df2 = df2[~(df2["State"] == "Puerto rico")]

In [124]:
Data = pd.merge(df , df2 , on=['City' , 'Code' , 'year'] , how='left')

In [125]:
Data.isnull().sum()

date                                        0
name                                        0
age                                         0
gender                                      0
armed                                       0
race                                        0
City                                        0
Code                                        0
flee                                        0
body_camera                                 0
signs_of_mental_illness                     0
police_departments_involved                 0
year                                        0
State                                     729
Median household income                   729
PercentOfPeopleBelowPovertyLevel          729
Percent high school graduate or higher    729
share_white                               729
share_black                               729
share_native                              729
share_asian                               729
share_hispanic                    

In [127]:
df.to_csv("Clean_fact.csv" , index=False)

In [128]:
df2.to_csv("Clean_Dim.csv",index=False)

In [129]:
Data.to_csv("Clean_Data.csv",index=False)